**EXEMPLO 02** <br>
Você recebeu um convie para uma consultoria, na qual deve desenvolver um modelo de previsões de feedbacks de clientes em produtos comprados na loja, que serão serão coletados do instagram

Os dados que você vai utilizar estão localizados em:
https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv

Na coluna 'review_title' você vai encontrar feedbacks passados dos nossos clientes em nossos produtos e, na coluna 'overall_rating', a nota que foi dada. Esse é o único dado que temos para auxilair na criação desse modelo de previsões.

In [83]:
# importar bibliotecas
# tensorflow é uma biblioteca de código aberto para aprendizado de máquina 
# e aprendizado profundopara treinamento e inferência de redes neurais artificiais.
# https://www.tensorflow.org/
# %pip install tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
import numpy as np
import pandas as pd

# constante do endereco do arquivo
ENDERECO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'


In [96]:
# obter dados
try:
    print('Obtendo dados...')
    df = pd.read_csv(ENDERECO_DADOS,sep=',',encoding='utf-8')
    print(df['reviewer_state'].value_counts())
    #print(df.shape)
    df = df[df['reviewer_state'] == "PR"][['review_text','overall_rating']] 
    
    
    #print(df.head()) 
    #print(df.shape)
    
    #print(df.columns)

    
    # delimitando colunas para criação do modelo
    df = df.dropna(subset=['review_text','overall_rating'])

    # transformando colunas em arrays
    texts = np.array(df['review_text'])
    ratings = np.array(df['overall_rating'])  
    
    #print(df.head())
    #print(len(df))
    #print(df['overall_rating'].unique())
    #print(df['overall_rating'].value_counts())

except Exception as e:
    print('Erro ao obter dados: ',e)


Obtendo dados...
reviewer_state
SP    49207
RJ    17580
MG    16322
PR     6745
RS     6622
BA     4806
SC     4478
ES     3052
PE     2955
GO     2532
DF     2395
CE     2171
PA     1497
MA     1218
MT      990
RN      926
PB      885
MS      834
AL      729
PI      699
SE      537
TO      397
RO      327
AM      208
AP      113
AC       87
RR       70
Name: count, dtype: int64


C:\Users\36131872024.1\AppData\Local\Temp\ipykernel_19880\2188012562.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ENDERECO_DADOS,sep=',',encoding='utf-8')


In [85]:

# Vetorização de texto
try:
    print('Vetorizando texto...')

    # Passo 1: Tokenizar os textos
    tokenizer = Tokenizer()
    # fit_on_texts: cria o vocabulário
    # vocabulário: dicionário que mapeia palavras para números
    # cada palavra é mapeada para um número inteiro e 
    # esse número inteiro é o índice da palavra no vocabulário
    # toda vez que uma palavra aparece em um texto,
    # ela é substituída pelo seu índice no vocabulário
    tokenizer.fit_on_texts(texts)

    # Converter os textos em sequências de números
    # texts_to_sequences: converte os textos em sequências de números
    # É a vetorização dos textos. Cada texto é convertido em um
    # vetor de números inteiros, onde cada número inteiro representa
    # uma palavra do texto, baseada no índice da palavra no vocabulário
    vetores = tokenizer.texts_to_sequences(texts)
    
    # Passo 2: Padronizar o comprimento das sequências (deixar todas com o mesmo tamanho)
    padded_vetores = pad_sequences(vetores)  # Deixar todas as sequências com tamanho 10

    # Verificar o resultado da tokenização e padding
    print(padded_vetores)

    print('Texto vetorizado com sucesso!')

except Exception as e:
    print('Erro ao vetorizar texto: ',e)

Vetorizando texto...
[[    0     0     0 ...   274     4   142]
 [    0     0     0 ...    31   152    15]
 [    0     0     0 ...    27    27    27]
 ...
 [    0     0     0 ...    53     3    53]
 [    0     0     0 ...     9    23 10568]
 [    0     0     0 ...   676    66   408]]
Texto vetorizado com sucesso!


In [110]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Embedding, LSTM, Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

# Definir as constantes para o modelo
try:
                print('Construindo a rede neural...')
                # tamanho do vocabulário
                VOCAB_SIZE = len(tokenizer.word_index) + 1  

                # Comprimento máximo da sequência
                MAX_SEQUENCE_LENGTH = padded_vetores.shape[1]

                # Tamanho do vetor
                VETOR_LENGHT = int(np.sqrt(VOCAB_SIZE))

                # Construir o modelo sequencial
                model = Sequential()

                # Camada de entrada
                model.add(
                        Embedding(
                                input_dim=VOCAB_SIZE, 
                                output_dim=VETOR_LENGHT, 
                                input_length=MAX_SEQUENCE_LENGTH
                                )
                )

                # Camada LSTM para capturar a sequência do texto
                # rodar primeiro somente com 128
                model.add(LSTM(128))

                # rodar com mais camadas
                # return_sequences: parâmetro que indica se a camada deve retornar
                # a sequência inteira para a próxima camada
                # sequencia inteira é a sequência de saídas de cada unidade de tempo
                #model.add(LSTM(32)) # SÓ UTILIZA O RETURN SEQUENCE SE ACRESCENTAR CAMADAS

                #model.add(LSTM(64, return_sequences=True))

                # última camada LSTM não precisa retornar a sequência inteira
                #model.add(LSTM(32))


                # Camada final para prever a nota (classificação)
                # Definir a quantidade de neurônios de acordo 
                # com a quantidade de classes/categorias
                model.add(Dense(5, activation='softmax'))

                # construir o modelo
                model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))

                # Usar um otimizador Adam com uma taxa de aprendizado menor
                # Adam: é um otimizador que ajusta a taxa de aprendizado
                otimizador = Adam(learning_rate=0.0001) 

                # Compilar o modelo
                # loss precisa ser ajustado de acordo com o tipo de problema
                # nesse caso, é um problema de classificação
                # sparse_categorical_crossentropy: função de perda para classificação
                # de múltiplas classes/categorias
                # sparse: indica que os rótulos são inteiros
                # categorical: indica que os rótulos são one-hot encoded, ou seja,
                # cada rótulo é uma categoria
                # crossentropy: é a função de perda que mede a diferença entre a distribuição
                # de probabilidade prevista pelo modelo e a distribuição de probabilidade real
                # nesse caso observa-se q a perda durante as épocas
                # Se a perda estiver diminuindo, o modelo está aprendendo
                # Se a perda estiver aumentando, o modelo não está aprendendo e pode estar
                # ocorrendo overfitting
                model.compile(optimizer=otimizador, loss='sparse_categorical_crossentropy')

                # Verificar o resumo do modelo
                #model.summary()
except Exception as e:
                print('Erro ao construir a rede neural: ',e)



Construindo a rede neural...


In [117]:
# treinar a rede neural
try:
                print('Treinando a rede neural...')

                # Dividir os dados em treino e teste (80% treino, 20% teste)
                from sklearn.model_selection import train_test_split
                X_train, X_test, y_train, y_test = train_test_split(
                                                padded_vetores, # textos vetorizados
                                                ratings, # notas
                                                test_size=0.2, # 20% dos dados para teste
                                                random_state=42) # seed para reprodução dos resultados

                # Ajuste temporário dos rótulos para o intervalo [0, 4] para o treinamento
                # Visto que a classificação começa em 1 e termina em 5
                # e a classificação começa em 0
                y_train_adjusted = y_train - 1
                y_test_adjusted = y_test - 1

                #peso
                pesos = len(df['overall_rating'])/df['overall_rating'].value_counts()

                # Aumentar o número de épocas para dar mais tempo ao modelo para aprender
                # Aumentar o tamanho do batch para acelerar o treinamento, porém,
                # pode ser que o modelo não aprenda bem
                # Reduzir o tamanho do batch para dar mais tempo ao modelo para aprender
                # e para evitar overfitting e aprender melhor
                model.fit(
                        X_train, # textos de treino 
                        y_train_adjusted, # notas de treino
                        epochs=6, # número de épocas 
                        batch_size=16, # tamanho do batch de treino
                        #class_weight = pesos.to_dict(),
                        validation_data=(X_test, y_test_adjusted)
                        )
                
except Exception as e:
                print('Erro ao treinar a rede neural: ',e)

Treinando a rede neural...
Epoch 1/6
332/332 ━━━━━━━━━━━━━━━━━━━━ 37s 112ms/step - loss: 0.4127 - val_loss: 1.5822
Epoch 2/6
332/332 ━━━━━━━━━━━━━━━━━━━━ 37s 112ms/step - loss: 0.3719 - val_loss: 1.6675
Epoch 3/6
282/332 ━━━━━━━━━━━━━━━━━━━━ 5s 106ms/step - loss: 0.3590

KeyboardInterrupt: 

In [114]:
# realizar previsões
try:
                print('Realizando previsões...')
                # Exemplos de novos textos para testar o modelo
                novos_textos = [
                    "Simplesmente adorei! O produto superou minhas expectativas em todos os aspectos. A qualidade é excelente e os resultados são visíveis em pouco tempo. A embalagem é prática e o atendimento ao cliente foi impecável. Recomendo a todos que buscam um produto eficaz e confiável!",
                    "Incrível! Estou impressionada com a eficácia deste produto. A textura é suave e o cheiro é maravilhoso. Usei diariamente e já notei grandes melhorias. Além disso, o custo-benefício é excelente. Sem dúvida, voltarei a comprar e indicarei para amigos. Muito satisfeita!",
                    "Perfeito! Este produto mudou minha rotina. A aplicação é fácil e rápida, e os resultados são surpreendentes. Senti a diferença logo nas primeiras semanas. A marca realmente se preocupa com a qualidade. Não consigo imaginar meu dia a dia sem ele. Altamente recomendado!"
                ]

                # Vetorizar e padronizar os novos textos
                novas_sequencias = tokenizer.texts_to_sequences(novos_textos)
                novas_sequencias_padded = pad_sequences(novas_sequencias)

                # Fazer previsões
                predicoes = model.predict(novas_sequencias_padded)

                # Formatar valores de saída
                np.set_printoptions(suppress=True, precision=4)

                # Exibir as previsões
                print("Previsões:\n", predicoes)
except Exception as e:
                print('Erro ao realizar previsões: ',e)

Realizando previsões...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Previsões: [[0.     0.     0.004  0.1313 0.8646]
 [0.     0.     0.0024 0.0816 0.916 ]
 [0.0165 0.0163 0.2403 0.5304 0.1965]]
